<a href="https://colab.research.google.com/github/sandhrabijoy/_/blob/master/Weather_Prediction_in_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import keras

In [4]:
#loading the data
from zipfile import ZipFile
data="https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path=keras.utils.get_file(origin=data,fname="jena_climate_2009_2016.csv.zip")
zipfile=ZipFile(zip_path)
zipfile.extractall()
csv="jena_climate_2009_2016.csv"

df=pd.read_csv(csv)